# Download Model

In [ ]:
!pip install simpletransformers

In [ ]:
!pip install gdown

In [ ]:
import gdown
# Download the zip file
file_id = '1n5FWtANhrjtrSH0AJcQDnMm5gaGwHCAC'
destination = 'downloaded_file.zip'
gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', destination, quiet=False)


# Unzip the downloaded file
import zipfile
import os

# Create a directory to extract the files
extracted_folder = 'model'
os.makedirs(extracted_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# List the contents of the extracted folder
print(os.listdir(extracted_folder))

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1n5FWtANhrjtrSH0AJcQDnMm5gaGwHCAC
From (redirected): https://drive.google.com/uc?export=download&id=1n5FWtANhrjtrSH0AJcQDnMm5gaGwHCAC&confirm=t&uuid=deabf8a5-28e9-4b9e-b342-789d2e8e0db0
To: /content/downloaded_file.zip
100%|██████████| 1.17G/1.17G [00:19<00:00, 59.4MB/s]


['vocab.txt', 'model.safetensors', 'training_args.bin', 'tokenizer_config.json', 'scheduler.pt', 'config.json', 'special_tokens_map.json', 'eval_results.txt', 'model_args.json', 'optimizer.pt']


# Example context for the model

The model only accepts txt documents now. Get this example context, download it on your computer and then you will be able to upload it into the folder when it asks you to upload a file. You can read the txt file to be able to ask the model questions about it.

In [ ]:
import gdown; gdown.download('https://drive.google.com/uc?export=download&id=12wGtCKahINcOeSFlvkyjvorbk1gPZFAa', 'context.txt', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=12wGtCKahINcOeSFlvkyjvorbk1gPZFAa
To: /content/context.txt
100%|██████████| 1.12k/1.12k [00:00<00:00, 1.20MB/s]


'context.txt'

# Use the Model

In [ ]:
import os
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from google.colab import files

def save_context(context, model_dir):
    os.makedirs(model_dir, exist_ok=True)
    context_file = os.path.join(model_dir, 'context.txt')
    with open(context_file, 'a') as file:
        file.write("\n" + context)

def load_contexts(model_dir):
    context_file = os.path.join(model_dir, 'context.txt')
    if not os.path.exists(context_file):
        return ""
    with open(context_file, 'r') as file:
        return file.read().strip()

def extract_text_from_text_file(txt_path):
    with open(txt_path, 'r') as file:
        text = file.read()
    return text.strip()

def main():
    while True:
        # Define the model directory
        model_dir = '/content/model'
        model_name = '/content/model'
        model_type = 'bert'

        provide_context = input("Provide context by uploading a document? (yes/no): ").strip().lower()

        if provide_context == 'yes':
            # Upload document
            uploaded = files.upload()
            if uploaded:
                txt_path = list(uploaded.keys())[0]
                extracted_text = extract_text_from_text_file(txt_path)
                save_context(extracted_text, model_dir)
            else:
                print("No file uploaded. Proceeding without additional context.")

        # Load context
        context = load_contexts(model_dir)
        print("\n--- Loaded Context ---")
        print(context)
        print("\n----------------------")

        # Initialize the model
        model_args = QuestionAnsweringArgs()
        model = QuestionAnsweringModel(
            model_type,
            model_name,
            args=model_args,
            use_cuda=False
        )

        while True:
            question = input("\nEnter your question: ")

            # Make predictions with the model
            to_predict = [
                {
                    "context": context,
                    "qas": [
                        {
                            "question": question,
                            "id": "0"
                        }
                    ]
                }
            ]

            predictions, _ = model.predict(to_predict)

            print("\n--- Predicted Answer ---")
            if predictions and len(predictions) > 0:
                answer_list = predictions[0]['answer'][:2] # Up to two answers per question
                print(answer_list)
            else:
                print("No answer found.")
            print("\n------------------------")

            next_action = input("(r)estart the process or (e)xit? (r/e): ").strip().lower()
            if next_action == 'r':
                break
            elif next_action == 'e':
                print("Exiting the program. Goodbye!")
                return
            else:
                print("Invalid option. Exiting.")
                return

if __name__ == "__main__":
    main()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
